In [1]:
_HOLE_BOUND = ((-10, 10), (-10, 10))  # Hole boundary -10 <= x <= 10, -10 <= y <= 10

GRID_SCALE = 1
GRID_RESOLUTION = 2
GRID_W = 60
GRID_BOUND = ((-GRID_W / 2, GRID_W / 2), (-GRID_W / 2, GRID_W / 2))

In [2]:
from src.prediction.rf_model_trainer import rf_ModelTrainer

model_trainer = rf_ModelTrainer(
    rf_drag_config=(100, 42),
    rf_max_temp_config=(100, 42),
    rf_avg_temp_config=(100, 42),
    grid_bound=GRID_BOUND,
    grid_bound_width=GRID_W,
    grid_resolution=GRID_RESOLUTION,
    grid_scale=GRID_SCALE,
    desired_variance=0.9,
)

rf_model = model_trainer.get_model()
pca_model = model_trainer.get_pca()

--------------------------------
| Model Trainer: Random_Forest |
--------------------------------
-------------------------------
| Training set found, loading |
-------------------------------


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\eaton\\Desktop\\Campus\\Capstone\\capstone_1\\model\\train\\random_forest\\5852b421-ac79-e778-fb6f-026f7f79ec31\\train_input_original.npy'

In [ ]:
import numpy as np

res = model_trainer.get_train_set()

if res is not None:
    x, y = res

    test = x[1:2]
    drag, std1 = rf_model[0].predict(test, return_std=True)
    avg_temp, std2 = rf_model[1].predict(test, return_std=True)
    max_temp, std3 = rf_model[2].predict(test, return_std=True)

    print("real: [0.29223466, 372.84985, 452.61221]")
    print("predicted: ", drag, avg_temp, max_temp)
    print("std: ", std1, std2, std3)

In [ ]:
from src.grid.visualize_points import visualize_points
from src.geometry.pattern_unit import (
    PatternUnit,
    Shape,
    PatternTransformation,
    PatternTransformationMatrix,
    Pattern,
)

pattern1 = Pattern(
    pattern_transformation_matrix=PatternTransformationMatrix(
        pattern_unit=PatternUnit(
            Shape(
                w=10,
                h=7.5,
                area_functions=[
                    lambda x, y: x**2 / 16 + y**2 / 9 <= 1
                    and x**2 + y**2 >= 4,  # Donut
                ],
            ),
            k=1 / GRID_RESOLUTION,
        ),
        pattern_transformation=PatternTransformation(
            "circular",
            dx=2.5,
            di=25,
            phi=np.pi / 7,
        ),
        pattern_bound=GRID_BOUND,
    ),
)

visualize_points(
    pattern1.pattern_matrix,
    k=1 / GRID_RESOLUTION,
)

pattern2 = Pattern(
    pattern_transformation_matrix=PatternTransformationMatrix(
        pattern_unit=PatternUnit(
            Shape(
                w=10,
                h=5,
                area_functions=[lambda x, y: x**2 / 9 + y**2 / 2 <= 1],
            ),
            k=1 / GRID_RESOLUTION,
        ),
        pattern_transformation=PatternTransformation(
            "grid",
            dx=10,
            di=10 / 2,
        ),
        pattern_bound=GRID_BOUND,
    ),
)
visualize_points(
    pattern2.pattern_matrix,
    k=1 / GRID_RESOLUTION,
)

In [ ]:
from src.prediction.to_rf_input import to_rf_input

ins = to_rf_input(
    pca_model,
    pattern_matrix=pattern1.pattern_matrix,
    bound=GRID_BOUND,
    resolution=1 / GRID_RESOLUTION,
)
ins2 = to_rf_input(
    pca_model,
    pattern_matrix=pattern2.pattern_matrix,
    bound=GRID_BOUND,
    resolution=1 / GRID_RESOLUTION,
)
print(ins.shape)
print(ins2.shape)

drag, std = rf_model[0].predict(ins, return_std=True)
avg_temp, std2 = rf_model[1].predict(ins, return_std=True)
max_temp, std3 = rf_model[2].predict(ins, return_std=True)

print("ins1 prediction")
print(f"prediction: {drag}, std: {std}")
print(f"prediction: {avg_temp}, std: {std2}")
print(f"prediction: {max_temp}, std: {std3}")

drag, std = rf_model[0].predict(ins2, return_std=True)
avg_temp, std2 = rf_model[1].predict(ins2, return_std=True)
max_temp, std3 = rf_model[2].predict(ins2, return_std=True)
print("ins2 prediction")
print(f"prediction: {drag}, std: {std}")
print(f"prediction: {avg_temp}, std: {std2}")
print(f"prediction: {max_temp}, std: {std3}")